# Problem Set 3: Network analysis of drug target data

Due: Monday, February 22nd, 2021, at 11:59pm

Submission: Please submit your completed Jupyter notebook, with code and written answers, to Canvas, with the filename pset3_SUNET (replacing SUNET with your SUNET ID). Please compile your notebook into web (.html) or document (.pdf) formats, making sure your lines of code/ouput don't get truncated (especially in the PDF!).

Developing a novel disease therapeutic can cost billions of dollars. A key step in drug development is to identify new compounds or FDA-approved drugs as potential disease treatments. Sometimes we understand the biological pathways associated with a given disease, and are looking for compounds that modulate specific targets. In these cases, we would like to develop computational approaches that decrease the cost of identifying drugs by inferring compound-target relationships through the use of publicly available data. This problem set will be an exercise in inferring novel compound indications using data from [Drug Target Commons](https://drugtargetcommons.fimm.fi/) (DTC), a crowd-sourced database of compound–target interaction profiles. 

First, we will create a network of compound-target relationships. Next we will perform basis network analysis, quantifying the centrality of nodes and identifying communities. Finally, we will walk you through the process of training a predictive model for inferring novel compound indications using an embedded representation of the graph.

In [33]:
import sys

print(sys.executable)
sys.path

/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8


['/Users/joshuatanner/Desktop/Trans_BMI/Problem_Sets/PSet3',
 '/Library/Frameworks/Python.framework/Versions/3.8/lib/python38.zip',
 '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8',
 '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/lib-dynload',
 '',
 '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages',
 '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/extensions',
 '/Users/joshuatanner/.ipython']

## Part 1: Generating the network

### 1.1 IC50 values
DTC records a variety of measures of compound-target bioactivity. We are providing you with all entries that include a measurement of [IC50](https://en.wikipedia.org/wiki/IC50) in the file *DTC_IC50.csv*. What is IC50? Why would a high IC50 value indicate that the compound likely cannot be used to inhibit the target?

YOUR WRITTEN ANSWER TO 1.1 HERE

IC50 is the half maximal inhibitory concentration, or the concentration of a drug that can reduce a biological process by 50%. A high IC50 value indicates that in order to achieve a 50% reduction, high amounts of the drug need to be administered, which is suboptimal.

### 1.2 Identify positive and negative compound-target pairs
Our first step is to use the IC50 values for each compound-target pair to determine if the compound can be used to inhibit the target. Some compound-target pairs have been assayed many times and therefore have many IC50 values recorded. Generate a list of compound-target pairs whose median IC50 value is <1000NM. These will be our positive hits. Generate another list of compound-target pairs whose median IC50 value is >= 5000NM. These will be our negative hits (in other words these compounds can not be effectively used to modulate the target). How many positive and negative hits do we have in our dataset?

We will use the following columns from the csv file:
 - compound_id
 - target_id
 - standard_value (this is the IC50 value)
 - compound_name (see Q1.5)
 - gene_names (see Q1.5)

In [28]:
# YOUR CODE FOR 1.2 HERE
import pandas as pd
from itertools import islice
import statistics

# Read in data to df, Trim out unneeded columns
total = pd.read_csv('data/DTC_IC50.csv',
                    usecols=['compound_id', 'target_id', 'standard_value', 'compound_name', 'gene_names'])
print(total[:10])

# Dictionary of compound-target pairs {(compound_id, target_id): [IC50]}
ct_pairs = {}
for ind in total.index:
    comp_targ = (total['compound_id'][ind], total['target_id'][ind])
    if comp_targ in ct_pairs:
        ct_pairs[comp_targ].append(total['standard_value'][ind])
    else:
        ct_pairs[comp_targ] = [total['standard_value'][ind]]
        
sanity = list(islice(ct_pairs.items(), 10))
print(f"\n\n\n {sanity}")


pos_hits = []
neg_hits = []

for key, value in ct_pairs.items():
    # If multiple values, change to median
    if len(value) > 1:
        median = statistics.median(value)
        ct_pairs[key] = median
    else:
        ct_pairs[key] = value[0]
    # List of negative hits
    if ct_pairs[key] >= 5000:
            neg_hits.append(median)
    # List of positive hits
    if ct_pairs[key] < 1000:
        pos_hits.append(median) 

        
sanity2 = list(islice(ct_pairs.items(), 10))
print(f"\n\n\n {sanity2}")

# Report length of both lists ^
print(f"\nneg_hits len: {len(neg_hits)}")
print(f"\npos_hits len: {len(pos_hits)}")
print(f"\ntotal pairs: {len(ct_pairs)}")

     compound_id compound_name target_id gene_names  standard_value
0  CHEMBL3545284  CERDULATINIB    Q9UHD2       TBK1            10.0
1  CHEMBL3545284  CERDULATINIB    Q9UHD2       TBK1            10.0
2  CHEMBL3545284  CERDULATINIB    Q9R117        NaN             0.5
3  CHEMBL3545284  CERDULATINIB    Q9P0L2      MARK1            10.0
4  CHEMBL3545284  CERDULATINIB    Q9P0L2      MARK1            10.0
5  CHEMBL3545284  CERDULATINIB    Q62137        NaN             8.0
6  CHEMBL3545284  CERDULATINIB    Q12851     MAP4K2            18.0
7  CHEMBL3545284  CERDULATINIB    Q08881        ITK           194.0
8  CHEMBL3545284  CERDULATINIB    Q08881        ITK           194.0
9  CHEMBL3545284  CERDULATINIB    Q00342        NaN            90.0



 [(('CHEMBL3545284', 'Q9UHD2'), [10.0, 10.0]), (('CHEMBL3545284', 'Q9R117'), [0.5]), (('CHEMBL3545284', 'Q9P0L2'), [10.0, 10.0]), (('CHEMBL3545284', 'Q62137'), [8.0]), (('CHEMBL3545284', 'Q12851'), [18.0]), (('CHEMBL3545284', 'Q08881'), [194.0, 194.

### 1.3 Build the network
Build a network of compound-target interactions using only the positive hits. We recommend using [networkx](https://networkx.github.io), since it is compatible with another Python package we will use in section 3. How many nodes and edges are in the graph?

There are some great [tutorials](https://networkx.github.io/documentation/stable/tutorial.html) on networkx that you can read to start building your networks!

In [35]:
# YOUR CODE FOR 1.3 HERE
!{sys.executable} -m pip install networkx
import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()
G.add_nodes_from(pos_hits)
nx.draw(G, with_labels=True, font_weight='bold')


  Using cached networkx-2.5-py3-none-any.whl (1.6 MB)
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


### 1.4 Add node types
To improve the descriptiveness of our network, let's add an attribute of "type" to each node to indicate whether the node is a compound or a target. Print the number of nodes of each type.

The function [set_node_attributes](https://networkx.github.io/documentation/stable/reference/generated/networkx.classes.function.set_node_attributes.html) and [get_node_attributes](https://networkx.github.io/documentation/stable/reference/generated/networkx.classes.function.get_node_attributes.html) will be helpful

In [23]:
# YOUR CODE FOR 1.4 HERE

### 1.5 Add node names
Now add an attribute "name" to each node using compound_name and gene_names from the DTC_IC50.csv file.

In [24]:
# YOUR CODE FOR 1.5 HERE

## Part 2: Exploring the network

### 2.1 Degree distribution
Plot a histogram showing the degree distribution of our graph. It might be helpful to log-scale the y-axis. Why are there so many nodes with degree 1?

Hint: the [degree](https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.degree.html) function will be helpful

In [25]:
# YOUR CODE FOR 2.1 HERE

YOUR WRITTEN ANSWER TO 2.1 HERE

### 2.2a Filter compounds by degree
Remove all compounds with degree == 1 from the graph. Print out the number of nodes and edges that remain, along with the number of compounds vs targets. Why might it be useful to filter out these very low degree compounds (think about the goal of this pset)?

In [26]:
# YOUR CODE FOR 2.2A HERE

YOUR WRITTEN ANSWER TO 2.2A HERE

### 2.2b Remove isolates
By removing compounds with degree == 1, we may have left some targets as isolates (not connected to any other nodes in the graph). Remove all isolates from the graph. Print out the number of nodes and edges that remain, along with the number of compounds vs targets. 

Hint: the [isolates](https://networkx.org/documentation/stable//reference/algorithms/isolates.html) function may be helpful

In [27]:
# YOUR CODE FOR 2.2B HERE

### 2.3 Connected components
What is a [connected component](https://en.wikipedia.org/wiki/Connected_component_(graph_theory))? How many are there in our graph? See [here](https://networkx.org/documentation/stable/reference/algorithms/component.html) for helpful networkx functions.

In [28]:
# YOUR CODE FOR 2.3 HERE

YOUR WRITTEN ANSWER TO 2.3 HERE

### 2.4 Visualize part of the network
Visualize the component containing the target P29375 (name=KDM5A). Color compounds in red and targets in yellow. Also display names for the targets. Use the networkx [draw](https://networkx.github.io/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw.html#networkx.drawing.nx_pylab.draw) function.

To help us with grading, please set the positioning of your visualization as follows:
```
pos = nx.spring_layout(subG, seed=888)
nx.draw(subG, pos=pos, ... any additional parameters)
```

To color your nodes, you can refer to this [post](https://stackoverflow.com/questions/13517614/draw-different-color-for-nodes-in-networkx-based-on-their-node-value) for help.

Tip: Setting the edge color to gray instead of black makes it easier to read the node labels.

In [29]:
# YOUR CODE FOR 2.4 HERE

### 2.5a Normalized degree centrality

We want to detect important nodes in our network. One way to characterize an important node is by *normalized degree centrality*, calculated for each node by dividing the number of neighbors by the total number of possible neighbors.

For this exercise, we want you to **hand-code** *normalized degree centrality* for our network and to set the *normalized degree centrality* as an attribute of your node. You may not use any predefined *degree_centrality* functions.

(HINT: remember that we are working with a bipartite graph. Think about the number of possible neighbors for each node.)

In [30]:
def normalized_degree_centrality(G, m, node):
    # INPUT:
    ## G: network
    ## m: number of possible neighbors
    ## node: node of interest to calculate degree centrality
    
    return ndc

# write code here to set the normalized_degree_centrality for each node

### 2.5b Normalization 

We see that normalized degree centrality depends on normalizing the number of neighbors by dividing by the total number of possible neighbors. How might this score be misleading? (HINT: Think in terms of the inbalance between the number of compounds & targets and experimental selection bias)

Plot a histogram of the distributions of normalized degree centrality colored by *type*. Please use the *density=True* and *log=True* attributes for the function. Why are the distributions so different?

You can refer to this [post](https://stackoverflow.com/questions/23617129/matplotlib-how-to-make-two-histograms-have-the-same-bin-width) for help with plotting two histograms in the same plot

In [31]:
# YOUR CODE FOR 2.5B HERE

YOUR WRITTEN ANSWER FOR 2.5B HERE

### 2.6 Betweeness centrality

*Betweeness centrality* is another method for determining important nodes in our network. While degree centrality only looks at directly connected edges, the betweeness centrality measures the importance of each node for holding the network together. It calculates the fraction of shortest paths that pass through a particular node. We will not make you code this part up since, as you can imagine, calculating shortest paths can be computationally expensive and requires optimization! For this part, use the [betweeness centrality algorithm for bipartite graphs](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.bipartite.centrality.betweenness_centrality.html) from the networkx package to calculate betweenness centrality for the component containing 'P29375' (not the entire graph). Set betweeness_centrality as an attribute of each node in the component.

In [32]:
# YOUR CODE FOR 2.6 HERE

### 2.7 Compare degree centrality and betweeness
Draw two plots of the component containing 'P29375' with targets labeled. Color one using degree centrality and the other using betweenness. Compare the two metrics - do they pick out the same nodes? How can a node have high betweeness but not degree centrality and vice versa? Note that we do not expect these two metrics to be on the same scale in this case since we computed one over the entire graph and one only over a connected component.

Hint: You can draw both plots using the same layout by first fixing the layout, and then passing it into the networkx draw method like this...

```
pos = nx.spring_layout(subG, seed=888)
nx.draw(subG, pos=pos, ... any additional parameters)
```

In [33]:
# YOUR CODE FOR 2.7 HERE

YOUR WRITTEN ANSWER FOR 2.7 HERE

### 2.8 Functional interpretation
Print out the compound in the full graph with the largest degree centrality. Print the genes associated with targets of this compound (we stored this information as the "name" of the targets). Input the associated genes into [Reactome Pathway Analysis](https://reactome.org/PathwayBrowser/#TOOL=AT) to find the functional pathways of these genes. Include a screenshot of the pathway browser output. Explain what functions the compound likely modulates.

In [34]:
# YOUR CODE FOR 2.8 HERE

YOUR WRITTEN ANSWER FOR 2.8 HERE

### 2.9 Community detection

Instead of looking at specific compounds or targets, we can also look at communities of nodes that are tightly connected. We will be using the Louvain algorithm [1] for community detection.

For networkx, we recommend you use the *community* package (https://python-louvain.readthedocs.io/en/latest/)

[1] Fast unfolding of communities in large networks, Vincent D Blondel, Jean-Loup Guillaume, Renaud Lambiotte, Renaud Lefebvre, Journal of Statistical Mechanics: Theory and Experiment 2008(10), P10008 (12pp)

For this exercise, please run louvain community detection over the full graph, using randomize=False (to help us with grading). Store the results as an attribute of each node. How many communities does the algorithm detect?

In [35]:
# YOUR CODE FOR 2.9 HERE

## 3 Network Embedding
Network embeddings assign nodes in a network to points in Euclidean space while still preserving network structure. This allows us to visualize the network in a new way and provides a feature space that is amenable to traditional machine learning. Node2vec [1] is a commonly used network embedding algorithm. We will use node2vec to generate feature representations for the edges in our graph. Then we will train a logistic regression model to predict new compound-target interactions.

[1] node2vec: Scalable Feature Learning for Networks. A. Grover, J. Leskovec. ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD), 2016. 

### 3.1 Prepare a subgraph
Our graph is large, so for the purposes of this pset, let's focus on the louvain community containing compound CHEMBL388978. We have provided CHEMBL388978_edges.txt, which contains the edges for the louvain community containing compound CHEMBL388978. Note that this file only contains  positive (ICD < 1000NM) edges. Read in the file and use it to generate a network using networkx. We will use this network for the rest of the assignment.

In [36]:
# YOUR CODE FOR 3.1 HERE

### 3.2 Remove test edges
Our goal is to predict new compound-target interactions. To evaluate how accurate our predictions are, we will use a set of test interactions contained in the file test_edges.txt. The first two columns represent the compounds and targets. The last column is a 1 if the interaction is positive (ICD < 1000NM) and a 0 if the interaction is negative (ICD > 5000NM). Remove the positive test edges from your network (we do not need to remove the negative test edges since our network only contains the positive edges). How many nodes and edges remain in your graph?

In [37]:
# YOUR CODE FOR 3.2 HERE

### 3.3 Create a node embedding
Use the [node2vec python package](https://github.com/eliorc/node2vec) to generate an embedded representation of your graph with default parameters (though note, parameter tuning can cause major differences in downstream analysis, so you'd definitely want to tune them in your research! Run PCA over the embedding and plot the first two principal components. Color compounds red and targets blue.

In [38]:
# YOUR CODE FOR 3.3 HERE

### 3.4 Create an edge embedding

Node2vec finds a vector to represent each node. However, in order to predict compound-target interactions, we need a vector representing each edge. We can apply a transformation to the node vectors to form edge vectors. Use the HadamardEmbedder from the Node2Vec package to do this.

For help, you can read this [tutorial](https://maelfabien.github.io/machinelearning/graph_5/#node-embedding)

In [39]:
# YOUR CODE FOR 3.4 HERE

### 3.5 Logistic regression
Using your edge embedding, train a logistic regression model to predict new edges in our graph. Use the scikit learn [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) package with default parameters. Again, you would want to tune these parameters in your own research, either through cross-validation or with a separate held-out validation set. Use the edges in train_edges.txt for training. Use the edges in test_edges.txt to evaluate performance. Plot the [ROC](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html) and [precision-recall](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html) curves for your model. Provide brief explanations for your results (think in terms of how your model is performing and how it's reflected in the curves).

In [40]:
# YOUR CODE FOR 3.5 HERE

YOUR WRITTEN ANSWER FOR 3.5 HERE